# Unit tests for hapCON
WIth Paths for UoC Server

In [1]:
### Some Code to set right paths (checks whether on Chicago cluster)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os as os
import sys as sys
import multiprocessing as mp
import sys
import socket as socket

socket_name = socket.gethostname()
print(socket_name)
if socket_name == "VioletQueen":
     path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket_name.startswith("midway2"):
     print("Midway jnovmbre partition detected.")
     path = "/project2/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
else: 
     raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)

print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

### If wanting to use local version and not  pip installed version
### Comment out to test installed version. Comment in to test development version:
### 

# Comment in if testing Developer Package
sys.path.insert(0,"./package/")  # hack to get local UoC Development package first in path
#sys.path.insert(0, "/mnt/archgen/users/yilei/tools/hapROH/package")  # Yilei's Development Version at MPI EVA

midway2-0401.rcc.local
Midway jnovmbre partition detected.
/project2/jnovembre/hringbauer/HAPSBURG
CPU Count: 28


# Test for 1240k panel on a Sardinian

In [6]:
# data preparation.
from hapsburg.PackagesSupport.hapsburg_run import hapCon_chrom_BFGS

path2bam="./Notebooks/Vignettes/Data/SUA001.bam"
path2bed1240k="./Notebooks/Vignettes/Data/chrX_1240k.bed"
outpath="./Notebooks/Vignettes/Data/SUA001.mpileup"

path2ref1240k="./Notebooks/Vignettes/Data/chrX_1240k.hdf5"
path2meta="./Notebooks/Vignettes/Data/meta_df_all.csv"

In [7]:
%%time
!module load samtools; samtools index $path2bam
!module load samtools; samtools mpileup --positions $path2bed1240k -r X -q 30 -Q 30 -o $outpath $path2bam

[mpileup] 1 samples in 1 input files
CPU times: user 12.9 ms, sys: 19.1 ms, total: 32 ms
Wall time: 589 ms


In [7]:
%%time 
conmle, _, _, _ = hapCon_chrom_BFGS(iid="SUA001", mpileup=outpath,
    h5_path1000g = path2ref1240k, meta_path_ref = path2meta)
assert(conmle > 0.1 and conmle < 0.105)

exclude 1033 sites outside the specified region
exclude 0 non-SNP sites
number of major reads at flanking sites: 10636
number of minor reads at flanking sites: 16
number of major reads at focal sites: 1291
number of minor reads at focal sites: 27
err rate at flanking sites: 0.001502
err rate at focal sites: 0.020486
saving sample as SUA001 in /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
estimated genotyping error by flanking sites: 0.001502
number of sites covered by at least one read: 3999, fraction covered: 0.085
hdf5 file saved to /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
finished reading mpileup file, takes 1.987.
number of sites covered by at least one read: 3999
hdf5 file saved to /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
estimated contamination rate: 0.102113(0.076802 - 0.127424)
CPU times: user 9.51 s, sys: 6.14 s, total: 15.7 s
Wall time: 15.2 s


In [8]:
%%time
!hapConX -m $outpath -r $path2ref1240k --meta $path2meta

exclude 1033 sites outside the specified region
exclude 0 non-SNP sites
number of major reads at flanking sites: 10636
number of minor reads at flanking sites: 16
number of major reads at focal sites: 1291
number of minor reads at focal sites: 27
err rate at flanking sites: 0.001502
err rate at focal sites: 0.020486
saving sample as SUA001 in /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
estimated genotyping error by flanking sites: 0.001502
number of sites covered by at least one read: 3999, fraction covered: 0.085
hdf5 file saved to /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
finished reading mpileup file, takes 1.809.
number of sites covered by at least one read: 3999
hdf5 file saved to /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
estimated contamination rate: 0.102113(0.076802 - 0.127424)
CPU times: user 676 ms, sys: 125 ms, total: 801 ms
Wall time: 28.6 s


### Test Running directly from bam file

In [9]:
%%time
# test running directly from bam file
conmle, _, _, _ = hapCon_chrom_BFGS(iid="SUA001", bam=path2bam,
                                    h5_path1000g = path2ref1240k, meta_path_ref = path2meta)
assert(conmle > 0.1 and conmle < 0.105)

exclude 1033 sites outside the specified region
exclude 0 non-SNP sites
total number of mapped reads: 14755
number of major reads at flanking sites: 10521
number of minor reads at flanking sites: 15
number of major reads at focal sites: 1291
number of minor reads at focal sites: 27
err rate at flanking sites: 0.001424
err rate at focal sites: 0.020486
saving sample as SUA001 in /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
estimated genotyping error by flanking sites: 0.001424
number of sites covered by at least one read: 3999, fraction covered: 0.085
hdf5 file saved to /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
finished reading bam file, takes 89.488.
number of sites covered by at least one read: 3999
hdf5 file saved to /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
estimated contamination rate: 0.102190(0.076878 - 0.127501)
CPU times: user 1min 23s, sys: 12.7 s, total: 1min 36s
Wall time: 1

# 2) Test 1000G panel with XiongNu DA43 sample

In [10]:
%%time
path2bam="./Notebooks/Vignettes/Data/DA43.bam"
path2bed1kg="./Notebooks/Vignettes/Data/chrX_1000G.bed"
!module load samtools; samtools index $path2bam
!module load samtools; samtools mpileup --positions $path2bed1kg -r X -q 30 -Q 30 -o ./Data/DA43.mpileup $path2bam

[mpileup] 1 samples in 1 input files
CPU times: user 44.5 ms, sys: 38.5 ms, total: 83 ms
Wall time: 2.24 s


In [11]:
%%time

conmle, _, _, _ = hapCon_chrom_BFGS(iid="DA43", bam=path2bam,
                                    h5_path1000g = path2ref1240k, meta_path_ref = path2meta)
assert(conmle > 0.02 and conmle < 0.025)

exclude 1033 sites outside the specified region
exclude 0 non-SNP sites
total number of mapped reads: 281907
number of major reads at flanking sites: 4776
number of minor reads at flanking sites: 41
number of major reads at focal sites: 569
number of minor reads at focal sites: 16
err rate at flanking sites: 0.008512
err rate at focal sites: 0.027350
saving sample as DA43 in /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/DA43.hdf5
estimated genotyping error by flanking sites: 0.008512
number of sites covered by at least one read: 5074, fraction covered: 0.108
hdf5 file saved to /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/DA43.hdf5
finished reading bam file, takes 77.717.
number of sites covered by at least one read: 5074
hdf5 file saved to /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/DA43.hdf5
estimated contamination rate: 0.021650(0.008590 - 0.034710)
CPU times: user 1min 13s, sys: 10.3 s, total: 1min 24s
Wall time: 1min 24s


# Test hapCon ROH

In [3]:
### Download Data (ca. 5gb)
# Commented out as it only runs from the shell
# !wget https://share.eva.mpg.de/index.php/s/Nsrd3kBCe8XSgcg/download/hapCon_ROH.zip

In [2]:
!hapCon_ROH -h

usage: hapCon_ROH [-h] [--mpileup MPATH] [--bamTable BAMTABLE] -i IID -r R
                  --meta META [-p PROCESSES] [--niter NITER] [--tol TOL]
                  [--prefix PREFIX]

Run hapCon_ROH from either mpileup or BamTable output

optional arguments:
  -h, --help           show this help message and exit
  --mpileup MPATH      Basepath to a list of mpileup file
  --bamTable BAMTABLE  Basepath to a list of BamTable file
  -i IID               IID of the target individual. Note that the filename of
                       mpileup output or BamTable should follow
                       $iid.chr$ch.mpileup or $iid.chr$ch.BamTable
  -r R                 path to reference panel hdf5 file.
  --meta META          path to the metadata for the reference panel.
  -p PROCESSES         Number of processes to use.
  --niter NITER        Maximum number of iterations.
  --tol TOL            Stopping criterion. If the estimated contamination
                       rates between two consecutive 

In [34]:
%%time
testbam = "./Data/hapCon_ROH/ROH_contam.test.bam"

!module load samtools; samtools index $testbam
!mkdir ./Data/mpileup
!module load samtools; for ch in {1..22}; do samtools mpileup --positions ./Data/hapCon_ROH/reference_1240k/1240kChr$ch.bed -r $ch -q 30 -Q 30 -o ./Data/mpileup/test.chr$ch.mpileup ./Data/hapCon_ROH/ROH_contam.test.bam; done


mkdir: cannot create directory ‘./Data/mpileup’: File exists
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
CPU times: user 701 ms, sys: 121 ms, total: 822 ms
Wall time: 34.4 s


In [35]:
%%time

metapath = "./Data/hapCon_ROH/meta_df_all.csv"
ref_files = "./Data/hapCon_ROH/reference_1240k/1240kChr"
pileups = "./Data/mpileup"

!hapCon_ROH --mpileup $pileups -i test -p 4 --prefix iter_ROH -r $ref_files --meta $metapath

saving hdf5 files in /project2/jnovembre/hringbauer/HAPSBURG/Data/hdf5
Traceback (most recent call last):
Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "<string>", line 1, in <module>
  File "<string>", line 1, in <module>
  File "/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
  File "/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
  File "/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
  File "/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
    exitcode = _main(fd)
  File "/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/multiprocessing/spawn.py", line 114, in _mai